In [2]:
import requests
from bs4 import BeautifulSoup
import re
import sys
import argparse
#from selenium import webdriver
from io import BytesIO

try:
    import pytesseract
except:
    !pip install pytesseract
    import pytesseract

try:
    from PIL import Image
except:
    !pip install PIL
    from PIL import Image

try:
    from selenium.webdriver import Chrome
except:
    !pip install selenium
    from selenium.webdriver import Chrome

pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract.exe'

In [3]:
driver = Chrome('chromedriver.exe')
link = 'http://observatorio.digemid.minsa.gob.pe/' # la url
driver.get(link)

content = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
medicamento = 'paracetamol'
# apronax
driver.implicitly_wait(10) #tiempo de espera para que carge y encuentre su elemento
driver.find_element_by_id('txtBuscador').send_keys(medicamento)

In [5]:
autocompletado = driver.find_elements_by_class_name('ui-menu-item') #guardamos todos los elemtos de esta lista ui-menu-item
autocompletado[0].click()
n_meds = len(autocompletado) #longitud
driver.find_element_by_id('btnBuscar').click() #estas buscando el boton de busqueda y le damos click

In [6]:
# aqui como hacerle click a un paracetamol
content = BeautifulSoup(driver.find_element_by_class_name('dataTables_scroll').get_attribute('innerHTML'), 'html.parser')
titulos = content.find(class_='dataTables_scrollHead').find('tr').select('th')
#saca contenido con beatiful, a esa elemento de datatables le saca el html
# extraemos los titulos, le doy un find tr' te devuelve un solo elemento estoy buscando un tr, 
dicts = []
keys = []
# inicializamos listas vacias y la otras las llaves
for tit in titulos:
    keys.append(tit.text)
# un encabezado es equivalente a un atributo
# en for voy a encontrar todos los titulos buscados
# 
rows = content.find(class_='dataTables_scrollBody').find(id='tbPrivados').select('tr')
# tr es una celda, 
for row in rows:
    data = []
    cols = row.select('td')
    for col in cols:
        data.append(col.text) #extraer el texto
    dicts.append(dict(zip(keys,data))) #zip me permite unir las dos listas en duplas o parejas, y dict me convierte en un dicccionario y lo guardamos en nuestra lista de diccionario
#creo una lista data    

In [7]:
dicts


[{},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '17/10/2019 03:39:32 p.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'ALIADA',
  'Precio Unit(S/.)': '27.54',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:26:19 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'FARMACIA LUCERO',
  'Precio Unit(S/.)': '35.00',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:22:16 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'BOTICA MILUZ',
  'Precio Unit(S/.)': '35.00',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:19:35 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'BOTICA MILUZ',
  'Precio Unit(

In [10]:
# aqui es como le diera click a todos los paracetamoles
# en este bucle for quiero iterar sobre todo los elementos menos uno pq ya entre al primero
# bucle para captar todas las opciones de paracetamol
for i in range(len(autocompletado)-1): #q vaya desde la longitud menos 1
    driver.find_element_by_class_name('ui-autocomplete-input').clear() #encuentra todos los paracemtamol, limpia el input
    driver.implicitly_wait(10)
    driver.find_element_by_class_name('ui-autocomplete-input').send_keys(medicamento) # le damos la palabra magica del paracetamol
    driver.find_elements_by_class_name('ui-menu-item')[i+1].click() #encontramos el objeto de la lista a la cual le damos el clikc, el 0 ya lo hemos visitado por eso es i+1
    driver.find_element_by_name('btn_buscar').click() #
    
    content = BeautifulSoup(driver.find_element_by_class_name('dataTables_scroll').get_attribute('innerHTML'), 'html.parser')
    rows = content.find(class_='dataTables_scrollBody').find(id='tbPrivados').select('tr')
#aqui nos centramos en filas
    for row in rows:
        data = []
        cols = row.select('td')
        for col in cols:
            data.append(col.text)
        dicts.append(dict(zip(keys,data))) #estoy añadiendo mas

In [11]:
dicts

[{},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '17/10/2019 03:39:32 p.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'ALIADA',
  'Precio Unit(S/.)': '27.54',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:26:19 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'FARMACIA LUCERO',
  'Precio Unit(S/.)': '35.00',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:22:16 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'BOTICA MILUZ',
  'Precio Unit(S/.)': '35.00',
  '': 'Ver'},
 {'Tipo Estab.': 'PRIVADO',
  'Fecha de Actualizac.': '20/11/2019 12:19:35 a.m.',
  'Producto': 'PARACETAMOL 10  mg/mL Solución para Perfusión',
  'Laboratorio': 'SANDERSON',
  'Farmacia/Botica': 'BOTICA MILUZ',
  'Precio Unit(